In [1]:
## FORMAT .MES FILES TO .CSV FILES ##

import os
import csv

def process_mes_file(input_file):
    output_rows = []
    with open(input_file, 'r') as file:
        lines = file.readlines()
        core_name = lines[4].strip()
        date = lines[53].strip()

        # Process um values
        um_values = lines[45:144]
        um_values = [float(value.strip()) for value in um_values]

        # Process percent values
        percent_values = lines[145:244]
        percent_values = [float(value.strip()) for value in percent_values]

        # Calculate individual percentages
        indiv_perc = [percent_values[0]]  # First value remains the same
        for i in range(1, len(percent_values)):
            indiv_perc.append(percent_values[i] - percent_values[i - 1])

        # Zip the data together
        for um, percent, indiv_percent in zip(um_values, percent_values, indiv_perc):
            output_rows.append([core_name, date, um, percent, indiv_percent])

    return output_rows

def write_to_csv(output_file, rows):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Core name", "Date", "um", "percent", "indiv perc"])
        writer.writerows(rows)

def process_folder(folder_path, output_folder):
    for filename in os.listdir(folder_path):
        if filename.endswith('.MES'):
            input_file = os.path.join(folder_path, filename)
            output_file = os.path.join(output_folder, os.path.splitext(filename)[0] + '.csv')
            rows = process_mes_file(input_file)
            write_to_csv(output_file, rows)
            print(f"Processed {filename} and saved as {output_file}")

# Example usage:
input_folder = 'input_path'
output_folder = 'output_path'
process_folder(input_folder, output_folder)

FileNotFoundError: [Errno 2] No such file or directory: 'input_path'

In [ ]:
#Grain size distribution plots of individual samples

import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import re
import cmcrameri

# Path to the folder containing CSV files
folder_path = "folder_path"

# Function to extract numbers from file names
def extract_numbers(filename):
    return [int(s) for s in re.findall(r'\d+', filename)]

# Initialize lists to store data from all files
um_all = []
indiv_perc_all = []
file_names = []  # Store file names for labels

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Read the CSV file
        data = pd.read_csv(os.path.join(folder_path, filename))
        
        # Extract the columns of interest
        um = data["um"]
        indiv_perc = data["indiv perc"]
        
        # Append data to the lists
        um_all.append(um)
        indiv_perc_all.append(indiv_perc)
        file_names.append(filename)  # Store file name

# Sort file names numerically
file_names_sorted = sorted(file_names, key=extract_numbers)

# Create individual plots for each file
for i, (um, indiv_perc, filename) in enumerate(zip(um_all, indiv_perc_all, file_names_sorted)):
    plt.figure(figsize=(4, 3))
    color = cmcrameri.cm.batlow(i / len(file_names_sorted))  # Use cmcrameri color scheme
    plt.plot(um, indiv_perc, alpha=0.8, color=color, linewidth=4)
    plt.title(f"Grain size distribution {filename}")  # Updated title
    plt.xlabel('Grain size (µm)')
    plt.ylabel('wt %')
    plt.grid(True)
    plt.xlim(1, 1000)  # Set the minimum and maximum values of the x-axis
    plt.xscale('log')
    plt.ylim(0, 0.1)
    plt.tight_layout()
    plt.savefig(os.path.join(folder_path, f'{filename}_plot.svg'), format='svg')  # Save individual plot

plt.show()  # Display combined plot in Python


In [ ]:
#Combined grain size distribution plot of the core

# Function to extract numbers from file names
def extract_numbers(filename):
    return [int(s) for s in re.findall(r'\d+', filename)]

# Initialize lists to store data from all files
um_all = []
indiv_perc_all = []
file_names = []  # Store file names for labels

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Read the CSV file
        data = pd.read_csv(os.path.join(folder_path, filename))
        
        # Extract the columns of interest
        um = data["um"]
        indiv_perc = data["indiv perc"]
        
        # Append data to the lists
        um_all.append(um)
        indiv_perc_all.append(indiv_perc)
        file_names.append(filename)  # Store file name

# Sort file names numerically
file_names_sorted = sorted(file_names, key=extract_numbers)

# Create a larger figure for combined plot
plt.figure(figsize=(15,9))

# Create a line plot for each file using cmcrameri's colormap
num_files = len(um_all)
color_palette = cmcrameri.cm.batlow  # Using 'roma' colormap from cmcrameri
for i, (um, indiv_perc, filename) in enumerate(zip(um_all, indiv_perc_all, file_names_sorted)):
    color = color_palette(i / num_files)
    plt.plot(um, indiv_perc, alpha=1, color=color, label=filename)  # Add label for the file
    
    # Set x-axis limit to start from 0 and end at 1000
    plt.xlim(1, 10000)

plt.title('Distribution of "um" vs "indiv perc"')
plt.xlabel('Grain size (µm)')  # Updated x-axis label
plt.ylabel('wt %')  # Updated y-axis label
plt.grid(True)

# Set x-axis to be logarithmic
plt.xlim(1, 1000)  # Set the minimum and maximum values of the x-axis
plt.xscale('log')
plt.ylim(0, 0.15)
# Display legend outside the plot in numeric order
plt.legend(sorted(file_names_sorted, key=lambda x: int(re.search(r'\d+', x).group())), bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()  # Adjust layout to prevent overlap

# Save the combined plot as .svg file
plt.savefig(os.path.join(folder_path, 'grainsize_distribution.svg'), format='svg')


In [ ]:
#Cumulative weight percent combine plots

# Function to extract numbers from file names
def extract_numbers(filename):
    return [int(s) for s in re.findall(r'\d+', filename)]

# Initialize lists to store data from all files
um_all = []
cum_perc_all = []
file_names = []  # Store file names for labels

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Read the CSV file
        data = pd.read_csv(os.path.join(folder_path, filename))
        
        # Extract the columns of interest
        um = data["um"]
        indiv_perc = data["indiv perc"]
        
        # Ensure the data is sorted by grain size
        sorted_indices = np.argsort(um)
        um = um.iloc[sorted_indices].reset_index(drop=True)
        indiv_perc = indiv_perc.iloc[sorted_indices].reset_index(drop=True)
        
        # Calculate cumulative percentage
        cum_perc = np.cumsum(indiv_perc)
        cum_perc = (cum_perc / cum_perc.iloc[-1]) * 100  # Normalize to 100%
        
        # Append data to the lists
        um_all.append(um)
        cum_perc_all.append(cum_perc)
        file_names.append(filename)  # Store file name

# Sort file names numerically
file_names_sorted = sorted(file_names, key=extract_numbers)

# Create a larger figure for combined plot
plt.figure(figsize=(15, 9))

# Create a line plot for each file using cmcrameri's colormap
num_files = len(um_all)
color_palette = cmcrameri.cm.roma  # Using 'roma' colormap from cmcrameri
for i, (um, cum_perc, filename) in enumerate(zip(um_all, cum_perc_all, file_names_sorted)):
    color = color_palette(i / num_files)
    plt.plot(um, cum_perc, linewidth=2, color=color, label=filename)  # Add label for the file
    
    # Set x-axis limit to start from 1 and end at 1000
    plt.xlim(1, 1000)

plt.title('Cumulative Distribution of "um" vs "cumulative perc"')
plt.xlabel('Grain size (µm)')  # Updated x-axis label
plt.ylabel('Cumulative wt %')  # Updated y-axis label
plt.grid(True)

# Set x-axis to be logarithmic
plt.xscale('log')
plt.ylim(0, 100)  # Assuming cumulative percentage goes from 0 to 100

# Display legend outside the plot in numeric order
plt.legend(sorted(file_names_sorted, key=lambda x: int(re.search(r'\d+', x).group())), bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()  # Adjust layout to prevent overlap

# Save the combined plot as .png file with transparency
plt.savefig(os.path.join(folder_path, 'cumulative_grainsize_distribution.png'), format='png', transparent=True)



In [ ]:
#Cumulative grain size distribution curves

for i, (um, cum_perc, filename) in enumerate(zip(um_all, cum_perc_all, file_names_sorted)):
    plt.figure(figsize=(3, 3))
    color = color_palette(i / num_files)  # Use cmcrameri colors for individual plots
    plt.plot(um, cum_perc, linewidth=2, color=color)
    plt.title(f"Cumulative Grain size distribution {filename}")  # Updated title
    plt.xlabel('Grain size (µm)')
    plt.ylabel('Cumulative wt %')
    plt.grid(True)
    plt.xlim(1, 1000)  # Set the minimum and maximum values of the x-axis
    plt.xscale('log')
    plt.ylim(0, 100)
    plt.tight_layout()
    plt.savefig(os.path.join(folder_path, f'{filename}_cumulative_plot.png'), format='png', transparent=True)  # Save individual plot

    plt.show()  # Display individual plot in Python

plt.show()  # Display combined plot in Python

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cmcrameri
import os
import pandas as pd

# Function to extract lower and upper depths from file name
def extract_depths(file_name):
    depths_str = file_name.split('_')[-1].split('-')
    lower_depth = float(depths_str[0])
    upper_depth = float(depths_str[1])
    return lower_depth, upper_depth

# Function to read data from multiple CSV files
def read_multiple_data(directory):
    all_data = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            data = pd.read_csv(file_path)
            lower_depth, upper_depth = extract_depths(os.path.basename(file_path).split('.')[0])  # Extract depths from file name
            data['Lower Depth'] = lower_depth
            data['Upper Depth'] = upper_depth
            all_data.append(data)
    return pd.concat(all_data, ignore_index=True)

def plot_combined_data(combined_data):
    fig, ax1 = plt.subplots(figsize=(2, 8))  # Adjust the width and height (in inches) as needed

    # Plot weight percent
    cmap = cmcrameri.cm.batlow  # Using the 'batlow' colormap from cmcrameri
    norm = mcolors.Normalize(vmin=0, vmax=0.1)  # Update the range to 0-0.1 for weight percent
    
    # Filter out NaN values
    combined_data_filtered = combined_data.dropna(subset=['um', 'Lower Depth', 'indiv perc'])

    sc = ax1.scatter(combined_data_filtered['um'], combined_data_filtered['Lower Depth'], c=combined_data_filtered['indiv perc'], cmap=cmap, norm=norm)
    cbar = plt.colorbar(sc, ax=ax1, label='Weight Percent')

    # Reverse the y-axis
    ax1.set_ylim(ax1.get_ylim()[::-1])

    # Set x-axis to log scale with range from 1 to 1000
    ax1.set_xscale('log')
    ax1.set_xlim(1, 1000)  # Set the x-axis limit from 1 to 1000

    # Set labels and titles
    ax1.set_xlabel('Grain Size (um)')
    ax1.set_ylabel('Depth (cm)')
    ax1.set_title('Combined Grain Size Distribution')

    # Group data by Lower Depth and find the micron with the highest weight percent
    max_um_per_depth = combined_data.groupby('Lower Depth').apply(lambda x: x.loc[x['indiv perc'].idxmax()]['um'])

    # Plot the result
    # Plot the result
    ax1.plot(max_um_per_depth.values, max_um_per_depth.index, color='gray', linestyle='-')
    
    plt.show()

# Directory containing CSV files
directory = "folder path"

# Read and combine data from multiple CSV files
combined_data = read_multiple_data(directory)

# Plot combined data
plot_combined_data(combined_data)
